In [210]:
import pdf2image
try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract
from pytesseract import Output

import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import numpy as np
import csv


In [10]:
#convert pdf to img PIL format
def pdf_to_img(pdf_file):
    return pdf2image.convert_from_path(pdf_file)
def ocr_core(file):
    text = pytesseract.image_to_string(file,lang='tha+tha4',config='-c preserve_interword_spaces=1')
    return text

# Detecting paragraphs and send to OCR


In [16]:
file_name = 'images/PO-im'
ext = '.jpg'

im = cv2.imread(file_name+ext)

img2gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(img2gray, (7,7), 0)

ret, mask = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,2))

dilation = cv2.dilate(mask,kernel,iterations = 1)



contours, hierarchy = cv2.findContours(dilation, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_SIMPLE)




for index,c in enumerate(contours):
    x,y,w,h = cv2.boundingRect(c)
    cv2.rectangle(im, (x, y), (x + w, y + h), (36,255,12), 2)
    
#     # crop image and send to OCR  , false detected will return no text 
    
#     cropped = im[y :y +  h , x : x + w]
    
#     print(index)

#     if not os.path.exists(file_name):
#         os.mkdir(file_name)
#     s = file_name + '/crop_' + str(index) + '.jpg' 
#     cv2.imwrite(s , cropped)
    
#     # ocr and save .txt
#     ocr_res = ocr_core(s)

#     t = file_name + '/txt_' + str(index)

#     with open(t, "w") as text_file:
#         text_file.write(ocr_res)


#     print(ocr_res)

    
cv2.imshow('thresh', mask)
cv2.imshow('dilation', dilation)
cv2.imshow('im', im)


cv2.waitKey(0)
cv2.destroyAllWindows() # destroys the window showing image
cv2.waitKey(1) #for mac OS

-1

### Test storing arrays
- Read the image, crop the area and send to ocr.  The result of ocr is the list of strings.  
- I also want to be able to locate the keywords on the image, so I save the cooridinate (x,y,w,h) with the strings in dictionary and store in .csv
- Then, I can search keywords in dictionary and highlight the boundary box on the image. 


In [271]:
file_name = 'images/PO-im'
ext = '.jpg'
invoice = cv2.imread(file_name+ext)
invoice_copy = cv2.imread(file_name+ext, cv2.IMREAD_GRAYSCALE)

# Apply contrast
(thresh, black_white_img) = cv2.threshold(invoice_copy, 127, 255, cv2.THRESH_BINARY)

# Apply Gaussian blur for smoothing to rid the image of artifacts
blur = cv2.GaussianBlur(black_white_img,(9,9),0)

# Apply threshold to bring contours to prominence
th = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 115, 1)

ret,thresh = cv2.threshold(th,127,255,0)

# Read our original invoice image to show the contours drawn in green
original_invoice_img = cv2.imread(file_name+ext)

# Find the contours in the image and store them in an array
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
invoice_contours = cv2.drawContours(original_invoice_img, contours, -1, (0, 255, 0), 3)

# Show the contours
# # cv2.imshow('Contours', invoice_contours)
# cv2.waitKey(0)
# cv2.destroyAllWindows() # destroys the window showing image
# cv2.waitKey(1) #for mac OS

# Iterate through our contours
index = 0
dt = {}
for index,c in enumerate(contours):
    x,y,w,h = cv2.boundingRect(c)
    cv2.rectangle(invoice_copy, (x, y), (x + w, y + h), (36,255,12), 2)
    
    # crop image and send to OCR  , false detected will return no text 
    
    cropped = invoice[y :y +  h , x : x + w]
    
    if not os.path.exists(file_name):
        os.mkdir(file_name)
    s = file_name + '/crop_' + str(index) + '.jpg' 
    cv2.imwrite(s , cropped)
    
    # ocr and save .txt
    raw_values_area = ocr_core(s).replace("  ", "").split('\n')
    
    ##cleaning
    for i,value in enumerate(raw_values_area):
        if value == '' or value == ' ' :
            ## store area, corr in dict
            raw_values_area.pop(i)
    
    corr = (x,y,w,h)
    
    if raw_values_area != []:
        dt[corr] = raw_values_area    


t = file_name + '/txt_raw' 

with open(t+'.csv', "w") as csv_file:
    writer = csv.writer(csv_file)
    for c, values in dt.items():
        writer.writerow([c, values])


cv2.imshow('crop', invoice_copy)
cv2.waitKey(0)
cv2.destroyAllWindows() # destroys the window showing image
cv2.waitKey(1) #for mac OS
#     print(raw_text_values)


-1

In [295]:
# search the coordinate of the keyword
def search_ocr(keyword,dict_):
    val_list = list(dt.values())
    key_list = list(dt.keys())
    corr_ls = []

    for area in dt.values():
        position = val_list.index(area)
        for sent in area:
            if keyword in sent:
                corr = key_list[position]
                if corr not in corr_ls:
                    corr_ls.append(key_list[position]) 
    return corr_ls

corr_ls_1 = search_ocr('ซื้อ',dt)

In [296]:
for x in corr_ls_1:
    print(x)

(0, 0, 595, 841)
(92, 535, 53, 13)
(459, 72, 77, 18)


In [297]:
# now, highlight keyword area found on the image
def find_area(corr_ls,file_):
    global image
    global cache
    image = cv2.imread(file_)
    
    cache = image.copy()
    # ignore first crop as it refers to the whole page
    invoice = cv2.imread(file_name+ext)
    
    for i,corr in enumerate(corr_ls[1:]):
        image = cache.copy()
        x,y,w,h = corr
        cv2.rectangle(image, (x, y), (x + w, y + h), (36,255,12), 2)
        cv2.imshow('crop', image)
        cv2.waitKey(0)
        cv2.destroyAllWindows() # destroys the window showing image
        cv2.waitKey(1) #for mac OS
    
find_area(corr_ls_1,'images/PO-im.jpg')

[ WARN:0@16189.897] global /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('images/PO-im/crop_145.jpg.jpg'): can't open/read file: check file path/integrity


## Read inside the table

In [124]:
file_name = 'images/PO-im'
ext = '.jpg'

im = cv2.imread(file_name+ext)

img2gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(img2gray, (5,5), 0)

ret, mask = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (4,2))

dilation = cv2.dilate(mask,kernel,iterations = 1)



contours, hierarchy = cv2.findContours(dilation, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_SIMPLE)




for index,c in enumerate(contours):
    if hierarchy[0][index][2] < 0 : #no child
        x,y,w,h = cv2.boundingRect(c)
        cv2.rectangle(im, (x, y), (x + w, y + h), (36,255,12), 2)
    elif hierarchy[0][index][3] < 0 : #no parents
        x,y,w,h = cv2.boundingRect(c)
        cv2.rectangle(im, (x, y), (x + w, y + h), (0,0,255), 2)
#     else :
#         x,y,w,h = cv2.boundingRect(c)
#         cv2.rectangle(im, (x, y), (x + w, y + h), (0,0,0), 2)
    
#     # crop image and send to OCR  , false detected will return no text 
    
#     cropped = im[y :y +  h , x : x + w]
    
#     print(index)

#     if not os.path.exists(file_name):
#         os.mkdir(file_name)
#     s = file_name + '/crop_' + str(index) + '.jpg' 
#     cv2.imwrite(s , cropped)
    
#     # ocr and save .txt
#     ocr_res = ocr_core(s)

#     t = file_name + '/txt_' + str(index)

#     with open(t, "w") as text_file:
#         text_file.write(ocr_res)


#     print(ocr_res)

    
cv2.imshow('thresh', mask)
cv2.imshow('dilation', dilation)
cv2.imshow('im', im)


cv2.waitKey(0)
cv2.destroyAllWindows() # destroys the window showing image
cv2.waitKey(1) #for mac OS

-1

In [80]:
hierarchy[0][1][3]

0

In [38]:
for component in zip(contours, hierarchy):
    currentContour = component[0]
    currentHierarchy = component[1]
    print(currentContour)
    print(currentHierarchy)

[[[ 26 683]]

 [[ 25 684]]

 [[ 25 825]]

 [[ 26 825]]

 [[ 27 826]]

 [[565 826]]

 [[567 824]]

 [[567 684]]

 [[566 683]]]
[[ 99  -1   1  -1]
 [ 37  -1   2   0]
 [  3  -1  -1   1]
 ...
 [359 357  -1 347]
 [360 358  -1 347]
 [ -1 359  -1 347]]
